In [26]:
import pandas as pd
import ast

import os
abspath = os.path.abspath(__file__)
dname = os.path.dirname(abspath)
os.chdir(dname)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 150)

print("done")

NameError: name '__file__' is not defined

In [2]:
#elist = "../data/editorials-1.jl"
#tags = "../data/by_article_110219.jl"
elist = "D:/Box Sync/editorials/data/editorials-1.jl"
tags = "D:/Box Sync/editorials/data/by_article_110219.jl"

def import_jl(filename):
    filename = filename
    full = open(filename, "r")
    list = []
    for x in full:
        list.append(x)
    return list

editorial_raw = import_jl(elist)
tags_raw = import_jl(tags)

In [3]:
print(editorial_raw[5])
print(tags_raw[5])

{"text": "\nHefty university fees are sending some grad students to food banks", "preview": "\n  \nAs costs rise, students and faculty members are pushing back\r\n  \n", "byline": "<p class=\"byline\">\n  By <a href=\"/author/katie-langin\">Katie Langin</a><time>Aug. 12, 2019</time></p>"}

{"headline": "A warning from the academic underground of adjuncts and contingent faculty", "tags": ["Read more Taken for Granted", "Taken for Granted", "Column", "Non-disciplinary"], "byline": "<p class=\"byline byline--article\">\n  By <a href=\"/author/beryl-lieff-benderly\">Beryl Lieff Benderly</a><time>Jun. 10, 2019 , 2:30 PM</time></p>"}



Next 2 blocks for initially processing jl input files. the blocks are for editorials and tags respectively 
in the final version of this, I want to have them both in the same function
an argument will indicate which file is being processeed. 
I want sidebar to be passed to the function, not inside it--as different scrapes could have different sidebars. However, to do this, I need to know how to make that argument optional, since sidebar will not be necessary for the tags file

In [15]:
def process(list, focus):
    
    list_temp1 = []
    for line in list:
        if "null" in line:
            line = line.replace("null", "\'null\'")
        line = ast.literal_eval(line.lower())
        list_temp1.append(line)
    list = list_temp1

    if focus == "editorial":
        editorial_dict = {"headline":[], "preview":[],"authors":[],"date":[] }

        sidebar = ["how to keep a lab notebook",
                "grad student unions dealt blow as proposed new rule says students aren’t ‘employees’",
                "what do we know about ph.d. scientists’ career paths?",
                "three lessons from industry that i’m taking back to academia"
                ]
        for line in list:
            if line["text"].replace("\n","")  in sidebar:
                if line["preview"] == "null":
                    continue

            editorial_dict["headline"].append(line["text"].replace("\n",""))
            editorial_dict["preview"].append(line["preview"].replace("\n","").replace("\r","").replace("  ",""))
            num_authors = (len(line["byline"].replace("<",">").split(">"))-5)//4
            au = line["byline"].replace("<",">").split(">")[4]
            for i in range(2,num_authors):
                au += ", " + line["byline"].replace("<",">").split(">")[i*4]
            editorial_dict["authors"].append(au)
            editorial_dict["date"].append(line["byline"].replace("<",">").split(">")[-5].replace(". ","-").replace(", ","-").replace(" ","").replace(",",""))

        editorial_df = pd.DataFrame(editorial_dict)
        editorial_df["date"] = pd.to_datetime(editorial_df.date, format='%b-%d-%Y')
        editorial_df.sort_values(by=["date"], inplace=True)
        out_df = editorial_df
        out_dict = editorial_dict
    
    elif focus == "tags":
        tags_dict = {"headline":[],"tags":[],"authors":[],"date":[],"time":[] }


        for line in list:
        # if "null" in line:
        #     line = line.replace("null", "\'null\'")
        #     line = ast.literal_eval(line)

            head = line["headline"].replace("\n","").replace("\"","")
            tags_dict["headline"].append(head)

            tags = line["tags"]
            unique_tags = []
            for tag in tags:
                if tag not in unique_tags:
                    unique_tags.append(tag)
            tags = unique_tags
            if len(tags) < 1:
                tags_dict["tags"].append("[]")
            elif len(tags[0])< 5:
                #tags_dict["tags"].append(", ".join(tags))
                tags_dict["tags"].append(tags)
            elif tags[0][:5] == "Read ":
                if tags[0][10:-1] not in tags[1]:
                    included = "false"
                    for tag in tags[2:]:
                        if tags[0][10:-1] in tag:
                            #tags_dict["tags"].append(", ".join(tags[1:]))
                            tags_dict["tags"].append(tags)
                            included = "true"
                            break
                    if included == "false":
                        tags[0] = tags[0][10:]
                        #tags_dict["tags"].append(", ".join(tags))
                        tags_dict["tags"].append(tags)

                else:
                    #tags_dict["tags"].append(", ".join(tags[1:]))
                    tags_dict["tags"].append(tags)
            else:
                #tags_dict["tags"].append(", ".join(tags))
                tags_dict["tags"].append(tags)

            num_authors = (len(line["byline"].replace("<",">").split(">"))-5)//4
            au = line["byline"].replace("<",">").split(">")[4]
            for i in range(2,num_authors):
                au += ", " + line["byline"].replace("<",">").split(">")[i*4]
            tags_dict["authors"].append(au)

            date_time = line["byline"].replace("<",">").split(">")[-5]
            time = date_time[-7:]
            date = date_time[:-9].replace(". ","-").replace(", ","-").replace(" ","").replace(",","")
            tags_dict["date"].append(date)
            tags_dict["time"].append(time)


        #print(tags_dict)

        tags_df = pd.DataFrame(tags_dict)
        tags_df.head()
        tags_df["date"] = pd.to_datetime(tags_df.date, format='%b-%d-%Y')
        tags_df["date"] = pd.to_datetime(tags_df["date"])
        tags_df.sort_values(by=["date"], inplace=True)
        out_df = tags_df
        out_dict = tags_dict
  
    return [out_df, out_dict]
 
out_edi = process(editorial_raw, "editorial")
out_tag = process(tags_raw, "tags")

edi_df = out_edi[0]
edi_dict = out_edi[1]

tag_df = out_tag[0]
tag_dict = out_tag[1]

In [23]:
print(len(edi_df))
print(len(editorial_raw))
print(len(tag_df))
print(len(tags_raw))
print(len(edi_df)-len(tag_df))

print("###################################################")

# print(edi_df.head())
# print("==================================================")
tag_df.head()

6312
7324
6136
6136
176
###################################################


,headline,tags,authors,date,time
5969,how to write a winning résumé,"[tooling up, advice, graduate, academic, industry, americas, united states]",peter fiske,1996-10-18,8:00 am
5968,the commandments of cover letter creation,"[tooling up, advice, americas]",peter fiske,1996-12-20,0:00 am
5974,his mother cried when he went into sales,"[tooling up, advice, graduate, postdoc, industry, biomedical, americas]",david g. jensen,1997-05-09,8:00 am
5973,what are headhunters and how do they work?,"[tooling up, advice, graduate, midcareer, postdoc, industry, biomedical, americas]",david g. jensen,1997-09-12,8:00 am
5996,dressing scientists for success: male case study,"[tooling up, advice, graduate, postdoc, academic, industry, americas]",peter fiske,1997-09-26,8:00 am


In [18]:
# I wonder why they're different numbers
print(len(edi_df))
print(len(tag_df))
print(len(edi_df)-len(tag_df))

#print(edi_df[edi_df["headline"] == "Sowing the Seeds of Change"])

# Dammit, I wonder why....

6312
6136
176


In [24]:
# merge dataframes by headline column

# still in progress needs a fuzzy join
# merge_test = editorial.merge(tags, how = "outer", left_on = "headline", right_on = "headline", indicator = True)

# print(len(merge_test))
# print(len(merge_test[merge_test["_merge"] != "both"]))
# merge_test[merge_test["_merge"] != "both"]


Generate means of processing sequential dates from the date of the first publication--basically, I hate the python dates function, and I'd rather use a function to go from sequential days to year instead of trying to extract the month and year

In [25]:
# Why am I doing this? I'm trying to calculate the months from the date_seq 
# Right now, this function returns a sequence from the actual start of the publications to the most recent publications
# I would want later versions of this to be more flexible--maybe allowing the start and end times to be set in the function?

def cumulative():
    months_r = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    months_l = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    # I need a list of cumulative days at the end of each month going forward
    cumulative_days = [31, 61, 92]
    cumulative_months = [13]

    for year in range(1997,2021):
        cumulative_months.append(cumulative_months[-1] + 12)
        if year%4 == 0:
            months = months_l
        else:
            months = months_r
        for month in range(1,13):
            #print(year, month)
            #print(months[month-1])
            cumulative_days.append(cumulative_days[-1]+months[month-1])
    return(cumulative_days, cumulative_months)

output = cumulative()

# print(cumulative()[0])
# print(output[1])